In [15]:
import pandas as pd
from tqdm import notebook

In [16]:
evaluated = pd.read_excel('gpt4o_eval.xlsx')
evaluated.drop('Unnamed: 0', axis=1, inplace=True)
EVALUATED = []
for row in evaluated.values.tolist():
    element = {}
    for cn, v in zip(evaluated.columns, row):
        element[cn] = v
    EVALUATED.append(element)

In [17]:
qa_dataset = pd.read_csv('QA_DATASET_in_progress_with_textbooks.csv')
qa_dataset_cols = qa_dataset.columns
qa_dataset.head(2)

,question,documents,correct_answer,ANSWER_IlyaGusev/saiga_llama3_8b,TIME_IlyaGusev/saiga_llama3_8b,relevant_documents,ANSWER_Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24,TIME_Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24,ANSWER_rscr/ruadapt_qwen2.5_32b:Q8_0,TIME_rscr/ruadapt_qwen2.5_32b:Q8_0,ANSWER_llama3.1:70b,TIME_llama3.1:70b
0,"Определите масштаб плана, если лес площадью 20...",NaN,NaN,<|start_header_id|>assistant<|end_header_id|>\...,14.534963,NaN,<|start_header_id|>assistant<|end_header_id|>\...,27.189879,Для определения масштаба плана нужно использов...,15.102432,"Чтобы определить масштаб плана, нам нужно вычи...",52.960524
1,"Определите масштаб карты, если улица длиной 2 ...",NaN,NaN,<|start_header_id|>assistant<|end_header_id|>\...,10.992461,NaN,<|start_header_id|>assistant<|end_header_id|>\...,17.908840,Чтобы определить масштаб карты по данному усло...,11.443384,"Ответ: Чтобы определить масштаб карты, нам нуж...",9.033217


In [18]:
QA_DATASET = []
for row in qa_dataset.values.tolist():
    element = {}
    for cn, v in zip(qa_dataset_cols, row):
        element[cn] = v
    QA_DATASET.append(element)

In [19]:
from numpy import nan as np_nan

In [20]:
import openai   

In [21]:
OPEN_AI_API_KEY = "sk-HF6u0UC240yl2r1TrAtyQ1KSAw8QTwpO"
LOCAL_LLM_HOST = "http://localhost:8000/v1"
LOCAL_LLM_KEY = "token-abc123"

In [22]:
llm_client = openai.OpenAI(
    base_url='https://api.proxyapi.ru/openai/v1',
    api_key=OPEN_AI_API_KEY
)

In [23]:
SYSTEM_PROMT = """
You are a highly capable evaluation assistant tasked with assessing student responses. 
For each answer, you will compare it against a provided reference answer, which serves as the standard for evaluation.
Your goal is to assign a score ranging from 0 to 10 based on the accuracy and completeness of the student's response. 
A score of 0 indicates that the answer is entirely incorrect and contradicts the reference, 
while a score of 10 signifies that the answer is flawless and fully aligns with the reference. 
You will provide only the score and no additional comments.
""".replace('\n', ' ')

In [26]:
QA_DATASET[0]

{'question': 'Определите масштаб плана, если лес площадью 20 га занимает на нём 20 см^2.',
 'documents': nan,
 'correct_answer': nan,
 'ANSWER_IlyaGusev/saiga_llama3_8b': '<|start_header_id|>assistant<|end_header_id|>\n\nДля определения масштаба плана необходимо использовать формулу масштаба:\n\nМасштаб = Размер объекта на плане / Размер объекта в реальности\n\nГде Размер объекта на плане — это 20 см^2 (площадь леса на плане), а Размер объекта в реальности — это 20 га (площадь леса в реальности).\n\n1 гектар (га) равен 10 000 квадратных метров. Таким образом, площадь леса в реальности составляет:\n\n20 га * 10 000 м^2/га = 200 000 м^2\n\nТеперь, используя формулу масштаба:\n\nМасштаб = Размер объекта на плане / Размер объекта в реальности\nМасштаб = 20 см^2 / 200 000 м^2\nМасштаб = 1 см^2 / 10 000 м^2\nМасштаб = 1:10 000\n\nТаким образом, масштаб плана составляет 1:10 000. Это означает, что на каждом сантиметре на плане соответствует 10 000 метров в реальности.<|eot_id|>',
 'TIME_IlyaG

In [28]:
RESULTS = []
for row in notebook.tqdm(QA_DATASET):
    if row['correct_answer'] is not np_nan and row['relevant_documents'] is np_nan and row['documents'] is np_nan:
        answers = [(k, v) for k, v in row.items() if k.startswith('ANS') and v is not np_nan]
        for model, answer in answers:
            flag = False
            for eval_row in EVALUATED:
                if model==eval_row.get('model') and answer==eval_row.get('model_answer') and row.get('question')==eval_row.get('question'):
                    RESULTS.append(eval_row)
                    flag = True
                    break
            if flag:
                continue
            messages = [{'role': 'system', 'content': SYSTEM_PROMT},
                        {'role': 'user', 'content': f"Question: {row['question']}\nReference answer: {row['correct_answer']}\nStudent answer:{answer}"}]
            completion = llm_client.chat.completions.create(
                model='gpt-4o',
                messages=messages,
                temperature=0.6,
                max_tokens=4096,
                stop=["</s>", "<eos>", "<|eot_id|>", "<|im_end|>"])
            RESULTS.append({'queston': row['question'], 
                            'reference_answer': row['correct_answer'], 
                            'model': model, 
                            'model_answer': answer,
                            'score': completion.choices[0].message.content})

  0%|          | 0/686 [00:00<?, ?it/s]

APITimeoutError: Request timed out.

In [30]:
df = pd.DataFrame(RESULTS)

In [32]:
df.to_excel('gpt4o_eval.xlsx')

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
counter_df = df[['model', 'score']].value_counts().to_csv('counter.csv')
counter_df = pd.read_csv('counter.csv')

In [71]:
counterdf = counter_df.pivot(index="model", columns="score", values="count")

In [72]:
counterdf

score,0,1,2,3,4,5,6,7,8,9,10
model,,,,,,,,,,,
ANSWER_IlyaGusev/saiga_llama3_8b,41.0,1.0,6.0,6.0,3.0,5.0,NaN,3.0,4.0,4.0,14.0
ANSWER_Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24,32.0,NaN,1.0,3.0,3.0,7.0,5.0,NaN,2.0,1.0,33.0
ANSWER_rscr/ruadapt_qwen2.5_32b:Q8_0,13.0,NaN,3.0,1.0,3.0,4.0,2.0,2.0,5.0,3.0,51.0
